### Init


Import the `cli` module that contains all the API commands and configure the API to connect to the server.


In [1]:
import liv4dcli as cli
from pandas import DataFrame

cli.config.url = "your.server.ca"

Create aliases to biomarkers sets and clinician user IDs for convenience.


In [2]:
bright = "Exudates,Cotton Wool Spots,Drusen,Uncertain - Bright"
red = "Microaneurysms,Hemorrhages,Sub-retinal hemorrhage,Pre-retinal hemorrhage,Neovascularization,Uncertain - Red"
disk = "Disk,Cup,Macula"
vessel = "Vessels - Uncertain"
lesions = "Exudates,Cotton Wool Spots,Drusen,Uncertain - Bright,Microaneurysms,Hemorrhages,Sub-retinal hemorrhage,Pre-retinal hemorrhage,Neovascularization,Uncertain - Red"

demo = "a8d42b15-4d1e-4a95-a95e-852b9712fc71"

### Users


List users:


In [ ]:
users = cli.user.list_user()
DataFrame.from_dict(users)

Add a clinician user:


In [ ]:
# cli.user.create(name="name",email="email",password="password", role='clinician')

Change the password of a user:


In [ ]:
# cli.user.update(demo, password='1234admin')

### Taches


List all tasks of a user:


In [ ]:
tasks = cli.task.list_task(demo)
DataFrame.from_dict(tasks)

Assign a task to a user:


In [ ]:
cli.task.create(10, demo)
cli.task.create([11, 12], demo, limit_biomarkers=red)
cli.task.create(13, demo, diagnostic=True, comment="Faire le diagnostic")

Delete all tasks of a user:


In [ ]:
cli.task.delete_all_from_user(demo)

### Revision


Read a revision:


In [ ]:
rev = cli.revision.get_revision(demo, 2)
print("ID", rev["id"])
print("Comment", rev["diagnostic"])
print("SVG", rev["svg"])

In [ ]:
rev = cli.revision.get_revision(demo, 11, svg=False)
print("Comment:", rev["diagnostic"])
rev = cli.revision.get_revision(demo, 13, svg=False)
print("Comment:", rev["diagnostic"])

Download a specific biomarker of a revision:


In [ ]:
b = cli.revision.get_biomarker(user_id=demo, image_id=2, biomarker="Hemorrhages")

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(b, cmap="gray")
plt.axis("off")

Modify a revision:


In [ ]:
# cli.revision.update(svg, demo, 2, "diagnostic")
cli.revision.update_diagnostic("diagnostic", demo, 2)

Modify a specific biomarker of a revision:


In [ ]:
import cv2

img = cv2.imread(...)
cli.revision.update_biomarkers(demo, 2, {vessel: img})

Completely delete a revision:


In [ ]:
# cli.revision.clear_revision(demo,2)

Transfer biomarkers from one clinician to another:


In [ ]:
cli.revision.transfer_biomarker(2, demo, prod, biomarker=vessel)  # Only the vessels
cli.revision.transfer_biomarker([10, 11], demo, prod)  # All biomarkers for images 10 and 11